In [4]:
import numpy as np
import kaggle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC

In [5]:
################################################################
# load corona data
def load_corona_data():
	train_x = np.load('../../Data/Corona/train_x.npy')
	train_y = np.load('../../Data/Corona/train_y.npy')
	test_x = np.load('../../Data/Corona/test_x.npy')
	return train_x, train_y, test_x
################################################################

In [6]:
train_x, train_y, test_x = load_corona_data()

In [7]:
kf = KFold(n_splits=5,shuffle=True, random_state=69)
models = []
rows = []

In [15]:
parameters = [['rbf',0.5,1,1],['poly',0.5,1,3],['poly',0.5,1,5],['linear',0.5,1,1],['rbf',0.5,0.01,1],['poly',0.5,0.01,3],['poly',0.5,0.01,5],['rbf',0.5,0.001,1],['poly',0.5,0.001,3],['poly',0.5,0.001,5],['rbf',0.05,1,1],['poly',0.05,1,3],['poly',0.05,1,5],['linear',0.05,1,1],['rbf',0.05,0.01,1],['poly',0.05,0.01,3],['poly',0.05,0.01,5],['rbf',0.05,0.001,1],['poly',0.05,0.001,3],['poly',0.05,0.001,5],['rbf',0.0005,1,1],['poly',0.0005,1,3],['poly',0.0005,1,5],['linear',0.0005,1,1],['rbf', 0.0005,0.01,1],['poly', 0.0005,0.01,3],['poly', 0.0005,0.01,5],['rbf', 0.0005,0.001,1],['poly', 0.0005,0.001,3],['poly', 0.0005,0.001,5]]

In [17]:
for i in range(len(parameters)):
    svm = SVC(kernel=parameters[i][0],C=parameters[i][1],gamma=parameters[i][2],degree=parameters[i][3])
    accuracy = []
    #K-Fold Cross Validation
    for train_index, test_index in kf.split(train_x):
        x_train, x_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        #Fit model on each fold
        svm.fit(x_train, y_train)
        #Predict on testing fold
        predict = svm.predict(x_test)
        predict = predict.reshape(len(predict),1)
        #Accuracy between prediction and true value for each K-Fold Validation
        accuracy.append(accuracy_score(y_test, predict))
    mean = np.average(accuracy)
    if(parameters[i][0] == 'rbf'):
        print("Kernel = " + parameters[i][0] + ", C = " + str(parameters[i][1]) + ", Gamma = " + str(parameters[i][2]))
        print("5-Fold Cross Validation Accuracy = " + str(mean))
    elif(parameters[i][0] == 'poly'):
        print("Kernel = " + parameters[i][0] + ", C = " + str(parameters[i][1]) + ", Gamma = " + str(parameters[i][2]) + ", Degree = " + str(parameters[i][3]))
        print("5-Fold Cross Validation Accuracy = " + str(mean))
    else:
        print("Kernel = " + parameters[i][0] + ", C = " + str(parameters[i][1]))
        print("5-Fold Cross Validation Accuracy = " + str(mean))
    models.append(mean)

Kernel = rbf, C = 0.5, Gamma = 1
5-Fold Cross Validation Accuracy = 0.6875816993464052
Kernel = poly, C = 0.5, Gamma = 1, Degree = 3
5-Fold Cross Validation Accuracy = 0.6509803921568628
Kernel = poly, C = 0.5, Gamma = 1, Degree = 5
5-Fold Cross Validation Accuracy = 0.677124183006536
Kernel = linear, C = 0.5
5-Fold Cross Validation Accuracy = 0.7516339869281046
Kernel = rbf, C = 0.5, Gamma = 0.01
5-Fold Cross Validation Accuracy = 0.6954248366013072
Kernel = poly, C = 0.5, Gamma = 0.01, Degree = 3
5-Fold Cross Validation Accuracy = 0.6875816993464052
Kernel = poly, C = 0.5, Gamma = 0.01, Degree = 5
5-Fold Cross Validation Accuracy = 0.6875816993464052
Kernel = rbf, C = 0.5, Gamma = 0.001
5-Fold Cross Validation Accuracy = 0.6875816993464052
Kernel = poly, C = 0.5, Gamma = 0.001, Degree = 3
5-Fold Cross Validation Accuracy = 0.6875816993464052
Kernel = poly, C = 0.5, Gamma = 0.001, Degree = 5
5-Fold Cross Validation Accuracy = 0.6875816993464052
Kernel = rbf, C = 0.05, Gamma = 1
5-Fold

In [19]:
final = SVC(kernel = 'linear', C = 0.5)

In [20]:
final.fit(train_x, train_y)

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [21]:
y_1 = final.predict(train_x)
y_1 = y_1.reshape(len(y_1),1)

In [24]:
accuracy_score(train_y, y_1)

0.788235294117647

In [25]:
import csv
def kaggleize(predictions,file):

	if(len(predictions.shape)==1):
		predictions.shape = [predictions.shape[0],1]

	ids = 1 + np.arange(predictions.shape[0])[None].T
	kaggle_predictions = np.hstack((ids,predictions)).astype(int)
	writer = csv.writer(open(file, 'w'))
	writer.writerow(['# id','Prediction'])
	writer.writerows(kaggle_predictions)

In [26]:
kaggleize(final.predict(test_x),'submission.csv')